<a href="https://colab.research.google.com/github/janbrederecke/analyzeD/blob/main/zindi_airquality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
! pip install -U layer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import layer
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
from lightgbm.sklearn import LGBMRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import phik

In [ ]:
sample_submission = layer.get_dataset("zindi/air-quality/datasets/sample_submission:1.1").to_pandas()
test_df = layer.get_dataset("zindi/air-quality/datasets/test:1.1").to_pandas()
train_df = layer.get_dataset("zindi/air-quality/datasets/train:1.1").to_pandas()

Output()

Output()

Output()

In [ ]:
#profile = ProfileReport(train_df, title = "Training data air quality", interactions = None )

In [ ]:
#profile.to_notebook_iframe()

In [ ]:

# phik_outcome = train_df.drop('ID', axis = 1).phik_matrix()
# phik_df = pd.DataFrame(phik_outcome.loc["pm2_5",:])
# phik_df["pm2_5"] = phik_df.pm2_5.astype("float")
# phik_df[(phik_df.pm2_5 > 0.1)].sort_values("pm2_5")

In [ ]:
def preprocessing(x):
    # Season
    # Use the date to create a season-variable
    x.date = pd.to_datetime(x.date)
    x['season'] = x.date.dt.month
    x['month'] = x['season']
    x['year'] = x.date.dt.year
    x['weekday'] = x.date.dt.weekday
    x['date2'] = x['date'].astype('category').cat.codes.astype('category')
    
    seasons = [1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 1]
    month_to_season = dict(zip(range(1,13), seasons))

    x.season = x.season.map(month_to_season)
    x.season = x.season.astype('category')
    #x.drop('date', inplace = True, axis = 1) 

    # Categorical variable recoding
    x['ID'] = x.ID.astype('str')
    x['device'] = x['device'].astype('category').cat.codes.astype('category')
    x['weekday'] = x['weekday'].astype('category').cat.codes.astype('category')
    x['month'] = x['month'].astype('category').cat.codes.astype('category')
    x['year'] = x['year'].astype('category').cat.codes.astype('category')

    # Select additional columns
    #solar_azimuth_angle = [col for col in x.columns if 'solar_azimuth_angle' in col]

    # Fill NA's

    
    columns = ['temp_mean',
               'UvAerosolIndex_solar_azimuth_angle',
               'CarbonMonoxide_CO_column_number_density',
               'NitrogenDioxide_tropospheric_NO2_column_number_density']

    x[columns] = x[columns].fillna(x.median())

    return x.loc[:, x.columns.isin(['ID',
                                    'device',
                                    'site_latitude',
                                    'site_longitude',
                                    'humidity',
                                    'temp_mean',
                                    'weekday',
                                    #'month', #seems to make the model worse than season
                                    'season',
                                    'year',
                                    'date2',
                                    'pm2_5',
                                    
                                    #'SulphurDioxide_solar_azimuth_angle',
                                    #'CarbonMonoxide_solar_azimuth_angle',
                                    #'NitrogenDioxide_solar_azimuth_angle',
                                    #'Formaldehyde_solar_azimuth_angle',
                                    'UvAerosolIndex_solar_azimuth_angle',
                                    #'Ozone_solar_azimuth_angle',
                                    #'Cloud_solar_azimuth_angle',
                                    'CarbonMonoxide_CO_column_number_density',
                                    'NitrogenDioxide_tropospheric_NO2_column_number_density'
                                ])]

# New section

# New section

In [ ]:
train_df = preprocessing(train_df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9923 entries, 0 to 9922
Data columns (total 14 columns):
 #   Column                                                  Non-Null Count  Dtype   
---  ------                                                  --------------  -----   
 0   ID                                                      9923 non-null   object  
 1   device                                                  9923 non-null   category
 2   site_latitude                                           9923 non-null   float64 
 3   site_longitude                                          9923 non-null   float64 
 4   humidity                                                9923 non-null   float64 
 5   temp_mean                                               9923 non-null   float64 
 6   CarbonMonoxide_CO_column_number_density                 9923 non-null   float64 
 7   NitrogenDioxide_tropospheric_NO2_column_number_density  9923 non-null   float64 
 8   UvAerosolIndex_solar_azimuth

In [ ]:
phik_outcome = train_df.drop('ID', axis = 1).phik_matrix()
phik_outcome

interval columns not set, guessing: ['site_latitude', 'site_longitude', 'humidity', 'temp_mean', 'CarbonMonoxide_CO_column_number_density', 'NitrogenDioxide_tropospheric_NO2_column_number_density', 'UvAerosolIndex_solar_azimuth_angle', 'pm2_5']


,device,site_latitude,site_longitude,humidity,temp_mean,CarbonMonoxide_CO_column_number_density,NitrogenDioxide_tropospheric_NO2_column_number_density,UvAerosolIndex_solar_azimuth_angle,pm2_5,season,year,weekday,date2
device,1.000000,1.000000,1.000000,0.452672,0.706137,0.131933,0.232104,0.167101,0.419022,0.303201,0.333250,0.000000,0.000000
site_latitude,1.000000,1.000000,1.000000,0.255570,0.649117,0.164512,0.186076,0.079336,0.279285,0.096775,0.231658,0.000000,0.424546
site_longitude,1.000000,1.000000,1.000000,0.301834,0.865403,0.198749,0.203429,0.096155,0.191231,0.046909,0.092976,0.000000,0.547607
humidity,0.452672,0.255570,0.301834,1.000000,0.791032,0.438333,0.410241,0.450517,0.261387,0.471904,0.350687,0.150036,0.881823
temp_mean,0.706137,0.649117,0.865403,0.791032,1.000000,0.188692,0.169021,0.231906,0.144617,0.356848,0.279562,0.075787,0.793516
CarbonMonoxide_CO_column_number_density,0.131933,0.164512,0.198749,0.438333,0.188692,1.000000,0.737359,0.350239,0.317038,0.306024,0.232161,0.207834,0.942790
NitrogenDioxide_tropospheric_NO2_column_number_density,0.232104,0.186076,0.203429,0.410241,0.169021,0.737359,1.000000,0.328155,0.212653,0.253059,0.124155,0.130725,0.805198
UvAerosolIndex_solar_azimuth_angle,0.167101,0.079336,0.096155,0.450517,0.231906,0.350239,0.328155,1.000000,0.230164,0.802899,0.566050,0.187674,0.999499
pm2_5,0.419022,0.279285,0.191231,0.261387,0.144617,0.317038,0.212653,0.230164,1.000000,0.563477,0.189054,0.049194,0.702469
season,0.303201,0.096775,0.046909,0.471904,0.356848,0.306024,0.253059,0.802899,0.563477,1.000000,0.927506,0.000000,1.000000


In [ ]:
phik_outcome['pm2_5'].sort_values(ascending = False)

pm2_5                                                     1.000000
date2                                                     0.702469
season                                                    0.563477
device                                                    0.419022
CarbonMonoxide_CO_column_number_density                   0.317038
site_latitude                                             0.279285
humidity                                                  0.261387
UvAerosolIndex_solar_azimuth_angle                        0.230164
NitrogenDioxide_tropospheric_NO2_column_number_density    0.212653
site_longitude                                            0.191231
year                                                      0.189054
temp_mean                                                 0.144617
weekday                                                   0.049194
Name: pm2_5, dtype: float64

In [ ]:
# Create feature columns
# Drop identifier column
X = train_df.drop(['ID', 'pm2_5'], axis = 1)

# Create target columns
y = train_df['pm2_5']

In [ ]:
pipelines = {
    'lgbm': make_pipeline(StandardScaler(),
                          LGBMRegressor(metric = "mae",)
                          )
}

In [ ]:
grid = {
    'lgbm': {
        'lgbmregressor__n_estimators':[100],
        'lgbmregressor__feature_fraction':[0.5, 0.8, 1],
        'lgbmregressor__max_depth':[3, 4, 5, 6, 7, 8, -1],
        'lgbmregressor__bagging_fraction':[0.635, 0.8, 1],
        'lgbmregressor__bagging_freq':[1, 0]      
    }
}

In [ ]:
# Create a blank dictionary to hold the models 
fit_models = {}
scores = []

# Loop through all the algos 
for algo, pipeline in pipelines.items():
    print(f'Training the {algo} model.')
    # Create new Grid Search CV Cclass 
    model = GridSearchCV(pipeline,
                         grid[algo],
                         n_jobs = -1,
                         cv = 10,
                         scoring = 'neg_mean_absolute_error'
                         )
    # Train the model 
    model.fit(X, y)
    # Store results inside of the dictionary
    fit_models[algo] = model
  
    scores.append({
        "model": algo,
        "best_score": model.best_score_,
        "best_params": model.best_params_
    })

Training the lgbm model.


In [ ]:
pd.options.display.max_colwidth = 200
pd.DataFrame(scores, columns = ["model", "best_score", "best_params"]).head()

,model,best_score,best_params
0,lgbm,-8.083417,"{'lgbmregressor__bagging_fraction': 0.635, 'lgbmregressor__bagging_freq': 0, 'lgbmregressor__feature_fraction': 0.8, 'lgbmregressor__max_depth': -1, 'lgbmregressor__n_estimators': 100}"


In [ ]:
test_df = preprocessing(test_df)
id = test_df.ID
test_df.drop(['ID'], axis = 1, inplace = True)

test_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,device,site_latitude,site_longitude,humidity,temp_mean,CarbonMonoxide_CO_column_number_density,NitrogenDioxide_tropospheric_NO2_column_number_density,UvAerosolIndex_solar_azimuth_angle,season,year,weekday,date2
0,17,0.289040,32.589580,0.756840,23.849653,0.025228,0.000032,-101.600266,4,0,4,43
1,23,0.356400,32.573000,0.810854,22.075347,0.035967,0.000023,-125.937157,1,0,0,109
2,26,0.381576,32.647109,0.883253,20.050694,0.033899,0.000023,-120.660301,4,0,0,88
3,9,0.379562,32.553714,0.782583,22.248958,0.048159,0.000025,-72.224289,4,0,5,16
4,18,0.344000,32.553000,0.770116,22.565217,0.033899,0.000023,-139.070473,1,0,1,117


In [ ]:
yhat_test = fit_models['lgbm'].predict(test_df)
air_quality = pd.DataFrame(yhat_test, columns = ["pm2_5"])["pm2_5"]
submission = pd.DataFrame([id, air_quality]).T
submission["ID"] = submission.ID.astype(str)
submission["pm2_5"] = submission.pm2_5.astype(float)
submission.head()

,ID,pm2_5
0,ID_UOH62J0XHX,47.217087
1,ID_4OPWSB0UHJ,83.041796
2,ID_3SLMNNG1Z3,58.596824
3,ID_MFUHTXYPM4,62.809975
4,ID_TY1DAND8ZP,94.567864


In [ ]:
submission.to_csv('submission.csv', index=False)